In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle

X = pickle.load(open('X.pickle', 'rb'))
y = pickle.load(open('y.pickle', 'rb'))

X = X/255.0   # normalize feature set

#### Optionally, to use an AMD GPU (with plaidml installed: https://github.com/plaidml/plaidml)

In [3]:
import os
os.environ['KERAS_BACKEND']='plaidml.keras.backend'

# When using plaidml, the libraries are imported from keras instead of tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D


Using plaidml.keras.backend backend.


#### Optionally, set up TensorBoard

In [6]:
from tensorflow.keras.callbacks import TensorBoard
import time
NAME = 'Cats-vs-dogs-cnn-32x3-{}'.format(int(time.time()))  # model name with timestamp
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))


#### Set up model and add layers

In [7]:
model = Sequential()

# first layer (Conv2D)
model.add(Conv2D(32, (3,3), input_shape=X.shape[1:])) #32 neurons, 3X3 window, shape50x50x1
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# second layer (Conv2D)
model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# third layer (Conv2D)
model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Fourth layer (Dense)
model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))

# output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam',
             metrics=['accuracy'])

# model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3)  
# batch_size: how many samples to pass each time, scale it relative to the size of total training set

# if using tensorboard:
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3, callbacks=[tensorboard]) 
# batch_size: how many samples to pass each time, scale it relative to the size of total training set

INFO:plaidml:Opening device "metal_amd_radeon_rx_vega_56.0"


Train on 17462 samples, validate on 7484 samples
Epoch 1/10
15008/17462 [========================>.....] - ETA: 2s - loss: 0.6644 - acc: 0.5818

KeyboardInterrupt: 

#### To view the tensorboard, use command:
 tensorboard --logdir=logs/

#### Save model

In [8]:
model.save('dog_vs_cat_3X32.model')